Take in SNPxSNP graph (from Cramer's V), and build graph.

In [22]:
def HL(df) : 

    ## imports
    import numpy as np
    import networkx as nx
    import scipy.stats as stats
    import pandas as pd
    import sklearn as sk
    import itertools
    import os
    from bed_reader import open_bed
    from IPython.display import SVG
    from sknetwork.embedding import LouvainNE
    from sknetwork.embedding import LouvainEmbedding
    from sknetwork.visualization import svg_graph, svg_bigraph
    from networkx.algorithms import bipartite
    from networkx.algorithms import community
    from scipy import sparse
    from sknetwork.hierarchy import LouvainHierarchy
    from sknetwork.hierarchy import cut_straight, dasgupta_score, tree_sampling_divergence
    from sknetwork.visualization import svg_graph, svg_bigraph, svg_dendrogram
    from sklearn.neighbors import NearestNeighbors
    from scipy.stats import chi2_contingency
    from scipy.sparse import diags


    # binarize
    dfSparse = sparse.csr_matrix(df) # convert data frame to a sparse matrix
    for i in range(np.shape(dfSparse)[0]):
        for j in range(np.shape(dfSparse)[1]):
            if dfSparse[i,j] <= 0.3:
                dfSparse[i,j] = 0
            else:
                dfSparse[i,j] = 1


    dfMatrix = dfSparse.toarray()
    dfMatrix.shape
    
    # get triangle
    triangle = np.triu(dfMatrix, k=1) # get upper half
    print(triangle)
    print(triangle.shape)
    triangleSparse = sparse.csr_matrix(triangle)

    # create adjacency matrix 
    G = nx.Graph(triangleSparse) # create graph from sparse data [UPDATE THIS]
    adjacency = nx.adjacency_matrix(G) # create Adjacency matrix
    adjacencyMatrix = sparse.csr_matrix(adjacency) # convert array to sparse row matrix form
    print(adjacencyMatrix.shape[0])

    ## train classifier and build graph
    louvain = LouvainNE(2)
    embedding = louvain.fit_transform(adjacencyMatrix)
    degrees = adjacency.dot(np.ones(adjacency.shape[0]))

    labels = []
    for i in range(adjacencyMatrix.shape[0]):
        labels.append(i)

    image = svg_graph(adjacencyMatrix, position=embedding, display_edge_weight= True, display_node_weight=True, 
    labels=labels, scale=1, node_order=labels, filename="graph", node_size_min=10, node_size_max=20, directed=False)

    #show graph
    return SVG(image)

    

In [23]:
    ## imports
    import numpy as np
    import networkx as nx
    import scipy.stats as stats
    import pandas as pd
    import sklearn as sk
    import itertools
    import os
    from bed_reader import open_bed
    from IPython.display import SVG
    from sknetwork.embedding import LouvainNE
    from sknetwork.embedding import LouvainEmbedding
    from sknetwork.visualization import svg_graph, svg_bigraph
    from networkx.algorithms import bipartite
    from networkx.algorithms import community
    from scipy import sparse
    from sknetwork.hierarchy import LouvainHierarchy
    from sknetwork.hierarchy import cut_straight, dasgupta_score, tree_sampling_divergence
    from sknetwork.visualization import svg_graph, svg_bigraph, svg_dendrogram
    from sklearn.neighbors import NearestNeighbors
    from scipy.stats import chi2_contingency
    from scipy.sparse import diags

In [28]:
#test = [[0, 0.54266815, 0.33628486, 0.85377528, 0.69203585],[0, 0, 0.2048833, 0.63337313, 0.77369248],[0, 0, 0., 0.44748536, 0.39128826],[0., 0., 0., 0., 0.81285984],[0., 0., 0., 0., 0.]]
test = [[0.00000000e+000,2.81838725e-077 ,1.05285636e-027, 2.67706557e-284, 1.41317825e-142] ,
[0.00000000e+000, 0.00000000e+000, 7.31277668e-011, 1.08470338e-112, 1.93160907e-198] ,
[0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 4.15164549e-050, 1.32033633e-037] ,
[0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 4.33336541e-235] ,
[0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000]] 
testSparse = sparse.csr_matrix(test)

In [29]:
HL(testSparse)

[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
(5, 5)
5


/Users/rishasurana/Library/Python/3.9/lib/python/site-packages/scipy/sparse/_index.py:103: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])


ValueError: The input matrix is empty.